In [ ]:
!pip install streamlit
!pip install streamlit_colab  # Allows Streamlit to run on Colab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement streamlit_colab (from versions: none)
ERROR: No matching distribution found for streamlit_colab


In [ ]:
# Ensure the necessary modules are installed
!pip install streamlit sqlite3

import sqlite3
import pandas as pd
import streamlit as st
import random

# Connect to SQLite database
conn = sqlite3.connect('timetable.db')
cursor = conn.cursor()

# Database Setup
# Create tables for Courses, Instructors, and Schedule if they don't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS Instructors (
                    instructor_id INTEGER PRIMARY KEY,
                    name TEXT NOT NULL
                 )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Courses (
                    course_id INTEGER PRIMARY KEY,
                    course_name TEXT NOT NULL,
                    instructor_id INTEGER,
                    total_hours INTEGER DEFAULT 42,
                    FOREIGN KEY (instructor_id) REFERENCES Instructors(instructor_id)
                 )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Schedule (
                    schedule_id INTEGER PRIMARY KEY,
                    course_id INTEGER,
                    instructor_id INTEGER,
                    day TEXT,
                    start_time TEXT,
                    end_time TEXT,
                    FOREIGN KEY (course_id) REFERENCES Courses(course_id),
                    FOREIGN KEY (instructor_id) REFERENCES Instructors(instructor_id)
                 )''')

conn.commit()

# Streamlit Interface
st.title("Timetable Generator")

# Section for adding instructors
st.header("Add Instructor")
instructor_name = st.text_input("Instructor Name")
if st.button("Add Instructor"):
    cursor.execute("INSERT INTO Instructors (name) VALUES (?)", (instructor_name,))
    conn.commit()
    st.success(f"Instructor {instructor_name} added successfully.")

# Section for adding courses and associating them with instructors
st.header("Add Course")
course_name = st.text_input("Course Name")
instructor_id = st.selectbox("Assign Instructor", [row[0] for row in cursor.execute("SELECT instructor_id FROM Instructors").fetchall()])
if st.button("Add Course"):
    cursor.execute("INSERT INTO Courses (course_name, instructor_id) VALUES (?, ?)", (course_name, instructor_id))
    conn.commit()
    st.success(f"Course {course_name} assigned to Instructor {instructor_id}.")

# Genetic Algorithm to Generate Timetable
def run_genetic_algorithm():
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    timeslots = ["10:00-11:00", "11:00-12:00", "12:00-1:00", "2:00-3:00", "3:00-4:00"]

    for course in cursor.execute("SELECT course_id, instructor_id FROM Courses WHERE total_hours > 0").fetchall():
        course_id, instructor_id = course

        for day in days:
            for slot in timeslots:
                cursor.execute("INSERT INTO Schedule (course_id, instructor_id, day, start_time, end_time) VALUES (?, ?, ?, ?, ?)",
                               (course_id, instructor_id, day, slot.split('-')[0], slot.split('-')[1]))
                cursor.execute("UPDATE Courses SET total_hours = total_hours - 1 WHERE course_id = ?", (course_id,))

                if cursor.execute("SELECT total_hours FROM Courses WHERE course_id = ?", (course_id,)).fetchone()[0] <= 0:
                    break
            conn.commit()

    st.success("Timetable Generated Successfully!")

# Button to run the genetic algorithm and generate the timetable
if st.button("Generate Timetable"):
    run_genetic_algorithm()

# Display the Timetable
st.header("Generated Timetable")
schedule_data = pd.read_sql_query("SELECT * FROM Schedule", conn)
st.write(schedule_data)

# Close connection
conn.close()


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


2024-11-12 14:10:33.076 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-12 14:10:33.471 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-12 14:10:33.477 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-12 14:10:33.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-12 14:10:33.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-12 14:10:33.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-12 14:10:33.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-12 14:10:33.505 Thread 'MainThread': mi